## IMDB, Amazon and Yelp review classification 
 - This project is for classification of different reviews of Imdb,Amazon and yelp.
 

In [1]:
# Importing dependencies
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


### a. Importing & pre-processing the dataset

In [2]:
# Load the dataset
data_yelp = pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [4]:
# showing few datasets
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
# assigning the names for columns
column_name = ['Review','Feedback']
data_yelp.columns = column_name

In [8]:
data_yelp.head() , data_yelp.shape

(                                              Review  Feedback
 0                           Wow... Loved this place.         1
 1                                 Crust is not good.         0
 2          Not tasty and the texture was just nasty.         0
 3  Stopped by during the late May bank holiday of...         1
 4  The selection on the menu was great and so wer...         1,
 (1000, 2))

In [12]:
# loading the amazon dataset
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
# assigining same column name for amazon daatset
data_amazon.columns = column_name
# showing head data after adding column names
data_amazon.head()

,Review,Feedback
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [13]:
data_amazon.shape

(1000, 2)

In [16]:
# loading Imdb dataset
data_Imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_Imdb.columns = column_name
data_Imdb.head()

,Review,Feedback
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [17]:
data_Imdb.shape

(748, 2)

In [19]:
# now ,we will append all the dataset into a single data
data = data_yelp.append([data_amazon,data_Imdb],ignore_index=True)

In [21]:
data.shape, data.head()

((2748, 2),
                                               Review  Feedback
 0                           Wow... Loved this place.         1
 1                                 Crust is not good.         0
 2          Not tasty and the texture was just nasty.         0
 3  Stopped by during the late May bank holiday of...         1
 4  The selection on the menu was great and so wer...         1)

In [22]:
# checking if there is any null value in data
data.isnull().sum()

Review      0
Feedback    0
dtype: int64

In [23]:
data['Feedback'].value_counts()

1    1386
0    1362
Name: Feedback, dtype: int64

In [24]:
x = data['Review']
y = data['Feedback']

### b. Data Cleaning
 - In this we will remove all our stopwords, punctuations and we will apply lemmatization

In [26]:
import string
punct = string.punctuation
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [27]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
print(stopwords)

['all', 'there', 'eleven', 'many', 'whatever', 'than', 'mostly', 'go', 'anyway', 'beforehand', 'used', 'hence', 'sometimes', 'hers', 'yourselves', 'neither', 'therein', 'side', 'although', 'about', 'only', 'nevertheless', 'nothing', 'six', '‘ll', 'less', '’s', 'onto', 'its', 'ours', 'namely', 'thereafter', 'in', 'towards', 'often', 'yourself', 'to', 'make', 'per', 'four', 'nowhere', 'something', 'everywhere', 'which', '‘s', 'amongst', 'everyone', 'thereby', 'and', 'how', 'however', 'might', '’re', 'had', 'whether', 'unless', 'yet', 'at', 'whoever', 'fifty', 'three', 'did', "'s", 'same', 'whom', 'eight', 'see', 'more', 'quite', 'put', 'on', 'the', 'nine', 'full', 're', 'around', 'us', '‘ve', 'others', 'everything', 'somehow', 'third', 'will', 'why', 'hereafter', 'we', 'via', 'afterwards', 'indeed', 'twelve', 'besides', 'moreover', 'was', 'done', 'or', 'must', 'most', 'anything', 'noone', 'their', 'otherwise', 'enough', 'get', 'your', 'last', 'seem', 'itself', 'herein', '‘m', 'been', 'va

In [28]:
import spacy 

nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_   # since there is no lemma for proper noun we are directly taking the lower case
    tokens.append(temp)
  
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [29]:
# example of data we cleaned
text_data_cleaning('good morning, it seems a very sunny day !')

['good', 'morning', 'sunny', 'day']

### c. TF-IDF vectorization

In [30]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [31]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [32]:
classifier = LinearSVC()

### d. Training our model
 - splitting our dataset into training and test set

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [35]:
x_train.shape, x_test.shape


((1923,), (825,))

In [36]:
x_train.head()

646                                       What a mistake.
2166    Damian is so talented and versatile in so many...
1686    The instructions didn't explain that a microph...
1368                                     Love This Phone.
47            The burger is good beef, cooked just right.
Name: Review, dtype: object

In [37]:
# fitting our x_train and y_train
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [38]:
clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000001B7FAD46D30>)),
                ('clf', LinearSVC())])

### e. prediction of our model

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
y_pred = clf.predict(x_test)

In [41]:
confusion_matrix(y_test,y_pred)

array([[311, 103],
       [ 77, 334]], dtype=int64)

In [45]:
accuracy_score(y_test,y_pred) *100  # we got a 78% accuracy !!

78.18181818181819